In [1]:
import pandas as pd
from datetime import datetime

import numpy as np

from pathlib import Path
import matplotlib.pylab as pl

from nna import visutils 

In [2]:

# %%
# CONFIGS
class pathMap():

    def __init__(self) -> None:
        scratch = '/scratch/enis/data/nna/'
        home = '/home/enis/projects/nna/'
        self.data_folder = scratch + 'database/'
        self.aggreated_data_cache_path = scratch + 'results/vis_export/'


#         self.exp_dir = '/home/enis/projects/nna/src/nna/exp/megan/run-3/'

        self.clipping_results_path = Path(scratch +
                                          'clipping_info/all-merged_2021-02-10')

        self.output_dir = scratch + 'real/'

        self.file_properties_df_path = self.data_folder + 'allFields_dataV5.pkl'

        self.results_folder = home + 'results/'
        self.vis_output_path = self.results_folder + 'vis/182tahb6-V1/'


def setup_configs():
    pathmap = pathMap()

    config = {}
    
    versiontag = '2wbqfywy-V1'
    config['versiontag'] = versiontag
    id2name={'1-0-0': 'biophony',
     '1-1-0': 'bird',
     '1-1-10': 'songbirds',
     '1-1-7': 'duck-goose-swan',
     '0-0-0': 'anthrophony',
     '1-3-0': 'insect',
     '1-1-8': 'grouse-ptarmigan',
     '0-2-0': 'aircraft',
     '3-0-0': 'silence'}
    
    
    generic_id2name = list(id2name.items())
    id2name = {}
    for k, v in generic_id2name:
        id2name[f'{versiontag}-{k}'] = v

    config['id2name'] = id2name
    config['input_data_freq'] = '10S'
    # FREQS to reduce results
    config['output_data_freq'] = '270min'

    classname2colorindex = {

        'anthrophony':14,
        'auto':15,
        'aircraft':9,

        'wind':18,
        'running water':0,
        'silence':1,

        'bird':4,
        'songbirds':5,
        'duck-goose-swan':16,
        'grouse-ptarmigan':17,

        'biophony':8,
        'insect':10,
        'mammal':2,

    }

    config['classname2colorindex'] = classname2colorindex
    config['prob2binary'] = True
    # cmap to use
    aCmap = pl.cm.tab20
    norm_cmaps = visutils.add_normal_dist_alpha(aCmap)

    config['norm_cmaps'] = norm_cmaps
    
    return pathmap, config




def setup(args, pathmap, region_location,):

    file_properties_df = pd.read_pickle(pathmap.file_properties_df_path)

    #important to keep them in order
    file_properties_df.sort_values(by=['timestamp'], inplace=True)

    # delete older than 2016
    fromtime = datetime(2016, 1, 1, 0)
    file_properties_df = file_properties_df[
        file_properties_df.timestamp >= fromtime]
    

    if not region_location:
        # region_location = [('anwr','49'),('prudhoe','11'),('prudhoe','26')]
        region_location = tuple(
            sorted(
                set(
                    zip(file_properties_df.region.values,
                        file_properties_df.locationId.values))))

    return region_location,file_properties_df
# %%
def sigmoid(data):
    return 1 / (1 + np.exp(-data))



In [3]:

def vis_preds_with_clipping_local(region_location, config, file_properties_df,
                            pathmap,norm_cmaps,cached_preds,save_fig=True):


    no_result = {}

    for region, location_id in region_location:
        # print(region, all_regions.index(region),'location_id',location_id)
        filtered_files = file_properties_df[file_properties_df.region == region]
        filtered_files = filtered_files[filtered_files.locationId ==
                                        location_id]
        filtered_files = filtered_files[filtered_files.durationSec > 0]

        figures_axes, no_result_paths = visutils.vis_preds_with_clipping(
            region,
            location_id,
            filtered_files,
            config['input_data_freq'],
            config['output_data_freq'],
            config['id2name'].keys(),
            norm_cmaps,
            pathmap.output_dir,
            pathmap.clipping_results_path,
            pathmap.vis_output_path,
            config['id2name'],
            clipping_threshold=1.0,
            prob2binary_flag=config['prob2binary'],
            pre_process_func=sigmoid,
            classname2colorindex=config.get('classname2colorindex',None),
            cached_pred=cached_preds[(region,location_id)],
            cached_pred_version='V1',
            save_fig=save_fig)


    return figures_axes, no_result_paths



In [4]:
region_location_years[85:]

NameError: name 'region_location_years' is not defined

In [5]:

region_location_years = tuple(
    sorted(
        set(
            zip(file_properties_df.region.values,
                file_properties_df.locationId.values,
                file_properties_df.year.values,))))

region_location_years = [i for i in region_location_years if 'stinchcomb' not in i[0]]


NameError: name 'file_properties_df' is not defined

In [ ]:
### FUNC from export_cli.py
def get_cached_file_name(config, df_type='prob'):

    csv_file_name_parts = (config['versiontag'],
                           f'prob2binary={config["prob2binary"]}',
                           f'output-data-freq={config["output_data_freq"]}',
                           df_type)
    csv_file_name = '_'.join(csv_file_name_parts)

    csv_file_name = csv_file_name + '.csv'

    return csv_file_name




In [6]:
%matplotlib agg
%matplotlib agg

pathmap, config = setup_configs()
region_location,file_properties_df = setup('', pathmap,region_location='Not None')

for region,location,year in region_location_years[85:]:
    print(region,location,year)
    if region!='' and location !='':
        region_location = [(region,location)]
    else:
        print('Region and location are not given, we will do all of them.')
        region_location = None


    csv_file_name =get_cached_file_name(config, df_type='prob')
    
    cached_preds = {}
    for region,location in region_location:
        cache_path = (Path(pathmap.aggreated_data_cache_path) / region / location / 'aggregated' / str(year))
        cached_preds[(region,location)]=(cache_path / csv_file_name)

    # cached_preds

    print('setup finished')

    # cached_preds=config['cached_preds']
    norm_cmaps=config['norm_cmaps'] 
    figures_axes, no_result_paths = vis_preds_with_clipping_local(region_location, config, file_properties_df, pathmap,norm_cmaps,cached_preds,save_fig=False)
    print('initial part')


    fig,ax=figures_axes[0]
    csv_path = 'dalton03labels.csv'
    (letters,y_multi_labels_by_month,
     x_multi_label_by_month, 
     classname2colorindex,color_indexes)=visutils.load_enis_labels4bars(csv_path,config.get('classname2colorindex',None))

    ### add bars

    # norm_cmaps=visutils.add_equally_spaced_bars(letters,y_multi_labels_by_month,
    #                         x_multi_label_by_month,ax,color_indexes,norm_cmaps)



    visutils.add_legend(ax,classname2colorindex,norm_cmaps,legend_ax_index=0,)
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)

    fig.savefig(f'vis2/{config["versiontag"]}_{region}_{location}_{year}.png')


NameError: name 'region_location_years' is not defined